In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math as math
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_log_error as rmsle
from sklearn.metrics import root_mean_squared_error as rmse
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.model_selection import KFold
import seaborn as sns
import itertools

In [10]:
train = pd.read_csv("train_comp.csv")
test = pd.read_csv("test_comp.csv")
# sample = pd.read_csv("sample_submission.csv")

train.head()

,Sex,Age,Height,Weight,Duration,Heart_Rate,Body_Temp,Calories
0,male,36,189,82,26,101,41.0,150
1,female,64,163,60,8,85,39.7,34
2,female,51,161,64,7,84,39.8,29
3,male,20,192,90,25,105,40.7,140
4,female,38,166,61,25,102,40.6,146


In [25]:
X = pd.get_dummies(train, columns=['Sex'], drop_first=True, dtype=int)
X = X.drop(columns=['Calories'])

y = np.log1p(train['Calories'])

X.head()

,Age,Height,Weight,Duration,Heart_Rate,Body_Temp,Sex_male
0,36,189,82,26,101,41.0,1
1,64,163,60,8,85,39.7,0
2,51,161,64,7,84,39.8,0
3,20,192,90,25,105,40.7,1
4,38,166,61,25,102,40.6,0


In [26]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=431)

# Train the linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions
predictions = model.predict(X_test)

predictions[predictions <= 0] = 1
predictions[predictions > 6] = 5

rmse(predictions, y_test)

0.18014809583855726

In [23]:
test.head()

,id,Sex,Age,Height,Weight,Duration,Heart_Rate,Body_Temp
0,750000,male,45,177,81,7,87,39.8
1,750001,male,26,200,97,20,101,40.5
2,750002,female,29,188,85,16,102,40.4
3,750003,female,39,172,73,20,107,40.6
4,750004,female,30,173,67,16,94,40.5


In [22]:
test_mod = pd.get_dummies(test, columns=['Sex'], drop_first=True, dtype=int)
test_mod = test_mod.drop(columns=['id'])

test_mod.head()

,Age,Height,Weight,Duration,Heart_Rate,Body_Temp,Sex_male
0,45,177,81,7,87,39.8,1
1,26,200,97,20,101,40.5,1
2,29,188,85,16,102,40.4,0
3,39,172,73,20,107,40.6,0
4,30,173,67,16,94,40.5,0


In [ ]:
# make the submission file! - Kaggle score 0.17876

pred = model.predict(test_mod)

submission = pd.DataFrame()

submission['id'] = np.arange(len(test))+750000
submission['Calories'] = np.expm1(pred)
submission.loc[submission['Calories'] < 0, 'Calories'] = 1
submission.loc[submission['Calories'] > 320, 'Calories'] = 300

submission.to_csv("LinReg_2.csv", index=False)

print(submission.head())
print()
print(submission['Calories'].describe())

       id    Calories
0  750000   33.588996
1  750001   98.554967
2  750002   91.311241
3  750003  139.659381
4  750004   79.064207

count    250000.000000
mean         89.678252
std          69.584215
min           2.234433
25%          31.756453
50%          70.658049
75%         136.102833
max         319.966628
Name: Calories, dtype: float64
